In [1]:
import pandas as pd
from fuzzywuzzy import process

/opt/anaconda3/envs/Medrag/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# HUMAN_REVIEWED_FILE = "example_plausible_gallstones.csv"
# # LLM_PREDICTIONS_FILE = "gpt_results_explanations_chol_partial.csv"
# LLM_PREDICTIONS_FILE = "gpt_results_explanations_chol_mini.csv"#"gpt_results_explanations_chol_v1_4o.csv"

HUMAN_REVIEWED_FILE = "First_human_merged_chol_report_mini.csv"
LLM_PREDICTIONS_FILE = "gallstone_gpt4.csv"

In [3]:
df_human = pd.read_csv(HUMAN_REVIEWED_FILE)
print(df_human.shape)
# display(df_human.describe().round(2))

for c in ['Interesting_hum', 'Int_ai', 'Plausible_hum',
       'Plaus_ai']:
    df_human[c] = pd.to_numeric(df_human[c]).astype(int)
display(df_human.nunique())
print(df_human.columns)
display(df_human)

(84, 8)


feature_name                 84
Interesting_hum               2
Int_ai                        2
Plausible_hum                 2
Plaus_ai                      2
COMMENTS                     44
Explanation (Interesting)    84
Explanation (Plausible)      84
dtype: int64

Index(['feature_name', 'Interesting_hum', 'Int_ai', 'Plausible_hum',
       'Plaus_ai', 'COMMENTS', 'Explanation (Interesting)',
       'Explanation (Plausible)'],
      dtype='object')


,feature_name,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai,COMMENTS,Explanation (Interesting),Explanation (Plausible)
0,"Long-standing illness, disability or infirmity...",1,1,1,1,CVD risk factors,Yes. This feature is interesting and novel bec...,Yes. Individuals with long-standing illnesses ...
1,Alanine aminotransferase,0,1,1,1,liver enzyme,Yes. Elevated alanine aminotransferase (ALT) l...,Yes. Elevated alanine aminotransferase (ALT) l...
2,Mouth/teeth dental problems Dentures,1,0,1,0,nutrition,No. This feature does not have a clear biologi...,No. The connection between dental issues and g...
3,Overall health rating Fair,0,1,0,1,NaN,Yes. A fair overall health rating indicates a ...,Yes. A fair overall health rating may indicate...
4,Standard PRS for alzheimer's disease (AD),1,0,1,0,NaN,No. Alzheimer's disease and gallbladder diseas...,No. There is no established mechanism linking ...
...,...,...,...,...,...,...,...,...
79,3-Hydroxybutyrate,0,0,0,0,High P value + low correlation,"No. The p-value is high (0.306), indicating th...",No. The p-value of 0.306 indicates that there ...
80,Hallux valgus (Bunion),0,0,0,0,High P value + low correlation,No. Hallux valgus (bunion) is primarily a musc...,No. There is no plausible mechanism or direct ...
81,Cholesterol in Chylomicrons and Extremely Larg...,1,0,0,1,- Our meta-analysis found insufficient eviden...,No. The relationship between cholesterol level...,Yes. Cholesterol metabolism is directly relate...
82,Job code at visit clerks,0,0,0,0,This is a not an ordinal data (i assume) - pr...,No. Job code is not likely to be directly rela...,No. The feature 'Job code at visit clerks' doe...


In [4]:
# display(df_human[['Interesting?',
#        'Plausible/Makes Sense?']].agg(["mean","sum","size"]).round(2))


display(df_human[['Interesting_hum', 'Int_ai', 'Plausible_hum',
       'Plaus_ai',]].agg(["mean","sum","size"]).round(2))


,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai
mean,0.55,0.14,0.46,0.3
sum,46.00,12.00,39.00,25.0
size,84.00,84.00,84.00,84.0


In [5]:
df_ai = pd.read_csv(LLM_PREDICTIONS_FILE).rename(columns={"feature_name":"feature"})
print(df_ai.shape)
display(df_ai.nunique())

display(df_ai.select_dtypes("number").agg(["mean","sum"]).round(2))
# # display(df_ai[['Interesting?',
# #        'Plausible/Makes Sense?']].agg(["mean","sum","size"]).round(2))
# print(df_ai["Interesting?"].value_counts())
# print(df_ai["Plausible/Makes Sense?"].value_counts())

df_ai.head()

(157, 8)


feature          157
target             1
novel              2
plausible          2
utility            1
novel_cot        157
plausible_cot    157
utility_cot      157
dtype: int64

,novel,plausible,utility
mean,0.78,0.3,0.0
sum,123.00,47.0,0.0


,feature,target,novel,plausible,utility,novel_cot,plausible_cot,utility_cot
0,Apolipoprotein A Blood biochemistry,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between Apolipoprotein A and ...,Apolipoprotein A (ApoA) is primarily associate...,To assess the utility of the feature 'Apolipop...
1,Number of self reported non cancer illnesses,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between the number of self-rep...,Cholelithiasis (gallstones) and gallbladder di...,To assess the utility of the feature 'Number o...
2,Fed up feelings,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The feature 'Fed up feelings' is not a commonl...,The feature 'Fed up feelings' may relate to th...,To assess the utility of the feature 'Fed up f...
3,(HT) hypertension genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between hypertension (HT) and...,To determine if there is a plausible explanati...,To assess the utility of the feature '(HT) hyp...
4,Home area population density urban or rural E...,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between home area population d...,To determine if the feature 'Home area populat...,To assess the utility of the feature 'Home are...


In [6]:
df_ai.loc[df_ai["feature"].isin(df_human["feature_name"])].shape[0]

20

In [7]:
df_human.loc[df_human["feature_name"].isin(df_ai["feature"])].shape[0]

20

### do fuzzy join

In [8]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# # Load the datasets
# first_df = pd.read_csv('/mnt/data/First_human_merged_chol_report_mini.csv')
# second_df = pd.read_csv('/mnt/data/gout_gpt4.csv')
first_df = df_human
second_df = df_ai

# Extract relevant columns
first_features = first_df[['feature_name']].dropna()
second_features = second_df[['feature']].dropna()

# Initialize a list to store match results
matches = []

# Perform fuzzy matching
for index, second_row in second_features.iterrows():
    feature = second_row['feature']
    match = process.extractOne(feature, first_features['feature_name'], scorer=fuzz.token_sort_ratio)
    if match:
        matches.append({
            'Second Feature': feature,
            'First Feature': match[0],
            'Match Score': match[1],
            'Second Index': index
        })

# Create a DataFrame from matches
matches_df = pd.DataFrame(matches)

# Filter matches with a score above a threshold (e.g., 80)
matches_df = matches_df[matches_df['Match Score'] >= 75]

# Merge the matched features back into the original dataframes
result = pd.merge(
    second_df,
    matches_df[['Second Index', 'First Feature', 'Match Score']],
    left_index=True,
    right_on='Second Index',
    how='left'
    # how='inner'
)

# Merge with the first dataframe to include additional columns if needed
final_result = pd.merge(
    result,
    first_df,
    left_on='First Feature',
    right_on='feature_name',
    # how='left'
    how='inner'
)

print(final_result.shape[0])
df_joint = final_result.drop_duplicates("feature_name").drop(columns=["Second Index",#"Match Score",
                                                                      "Explanation (Interesting)",
                                                                      "First Feature","Explanation (Plausible)"],errors="ignore")
print(df_joint.shape[0])

# # # Output the merged result
# df_joint.to_csv('fuzzy_merged_result.csv', index=False)

df_joint

54
47


,feature,target,novel,plausible,utility,novel_cot,plausible_cot,utility_cot,Match Score,feature_name,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai,COMMENTS
0,Apolipoprotein A Blood biochemistry,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between Apolipoprotein A and ...,Apolipoprotein A (ApoA) is primarily associate...,To assess the utility of the feature 'Apolipop...,100.0,Apolipoprotein A Blood biochemistry,1,0,1,0,NaN
1,Home area population density urban or rural E...,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between home area population d...,To determine if the feature 'Home area populat...,To assess the utility of the feature 'Home are...,100.0,Home area population density - urban or rural ...,1,0,1,0,Is this an ordinal parameter
2,(CED) coeliac disease genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between coeliac disease and ga...,Coeliac disease is an autoimmune condition tha...,To assess the utility of the feature '(CED) co...,94.0,Genetic risk for coeliac disease (CED),0,0,0,0,High P value
3,Time spend outdoors in summer,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between time spent outdoors a...,The question asks whether there is a plausible...,To assess the utility of the feature 'Time spe...,100.0,Time spend outdoors in summer,1,0,1,0,May reflect more physical activity
4,Concentration of Very Large VLDL Particles,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between very large VLDL parti...,To determine if there is a plausible explanati...,To assess the utility of the feature 'Concentr...,78.0,Concentration of IDL Particles,1,0,1,1,NaN
5,PSO psoriasis genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between psoriasis (PSO) and ga...,To determine if there is a plausible explanati...,To assess the utility of the feature 'PSO psor...,93.0,Genetic risk for psoriasis (PSO),0,0,0,0,High P value + low correlation
6,Concentration of Very Large HDL Particles,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between high-density lipoprot...,To determine if there is a plausible explanati...,To assess the utility of the feature 'Concentr...,85.0,Concentration of HDL Particles,1,0,1,1,NaN
7,Townsend deprivation index at recruitment,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The Townsend deprivation index is a measure of...,The Townsend deprivation index is a measure of...,To assess the utility of the 'Townsend depriva...,100.0,Townsend deprivation index at recruitment,1,1,1,1,important
8,Leg fat free mass,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The relationship between leg fat-free mass and...,Cholelithiasis and gallbladder diseases are of...,To assess the utility of the feature 'Leg fat-...,100.0,Leg fat-free mass,1,1,0,1,NaN
9,Mouth teeth dental problems Dentures,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,"The association between dental problems, parti...",The feature 'Mouth teeth dental problems Dentu...,To assess the utility of the feature 'Mouth te...,100.0,Mouth/teeth dental problems Dentures,1,0,1,0,nutrition


In [9]:
df_joint.sort_values(['Match Score'])

,feature,target,novel,plausible,utility,novel_cot,plausible_cot,utility_cot,Match Score,feature_name,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai,COMMENTS
4,Concentration of Very Large VLDL Particles,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between very large VLDL parti...,To determine if there is a plausible explanati...,To assess the utility of the feature 'Concentr...,78.0,Concentration of IDL Particles,1,0,1,1,NaN
38,HEIGHT height genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The association between height and gallbladder...,To determine if there is a plausible explanati...,To assess the utility of the feature 'HEIGHT h...,80.0,Enhanced Genetic risk for height (HEIGHT),0,0,0,0,"similar to row 65 - feature used twice, by mis..."
41,Water intake,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The relationship between water intake and gall...,Water intake is generally associated with hydr...,To assess the utility of the feature 'Water in...,82.0,Tea intake,1,0,1,0,Mid P value
18,Hot drink temperature Warm,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The association between 'Hot drink temperature...,The relationship between 'Hot drink temperatur...,To assess the utility of the feature 'Hot drin...,82.0,Hot drink temperature Hot,0,0,0,0,NaN
50,"dietary changes in the last 5 years Yes, becau...",(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,0,1,0,The association between dietary changes and ga...,Dietary changes can significantly impact the f...,To assess the utility of the feature 'dietary ...,82.0,"dietary changes in the last 5 years Yes, becau...",0,0,1,1,change of diet can be post cholelthiasis diagn...
6,Concentration of Very Large HDL Particles,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The relationship between high-density lipoprot...,To determine if there is a plausible explanati...,To assess the utility of the feature 'Concentr...,85.0,Concentration of HDL Particles,1,0,1,1,NaN
10,BD bipolar disorder genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The association between bipolar disorder (BD) ...,To determine if there is a plausible explanati...,To assess the utility of the feature 'BD bipol...,89.0,Genetic risk for bipolar disorder,1,0,1,0,less imagery?
23,OP osteoporosis genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The association between osteoporosis (OP) gene...,To determine if there is a plausible explanati...,To assess the utility of the feature 'OP osteo...,93.0,Genetic risk for osteoporosis (OP),0,0,0,0,NaN
5,PSO psoriasis genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between psoriasis (PSO) and ga...,To determine if there is a plausible explanati...,To assess the utility of the feature 'PSO psor...,93.0,Genetic risk for psoriasis (PSO),0,0,0,0,High P value + low correlation
2,(CED) coeliac disease genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between coeliac disease and ga...,Coeliac disease is an autoimmune condition tha...,To assess the utility of the feature '(CED) co...,94.0,Genetic risk for coeliac disease (CED),0,0,0,0,High P value


In [10]:
df_joint.select_dtypes("number").agg(["mean","sum"]).round(2)

,novel,plausible,utility,Match Score,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai
mean,0.83,0.3,0.0,95.85,0.57,0.15,0.55,0.34
sum,39.00,14.0,0.0,4505.00,27.00,7.00,26.00,16.00


In [11]:
df_joint.select_dtypes("number").corr().round(2)

,novel,plausible,utility,Match Score,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai
novel,1.00,-0.45,NaN,-0.10,-0.05,-0.45,-0.29,-0.51
plausible,-0.45,1.00,NaN,-0.10,0.09,0.38,0.30,0.32
utility,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Score,-0.10,-0.10,NaN,1.00,0.15,0.24,0.06,0.01
Interesting_hum,-0.05,0.09,NaN,0.15,1.00,0.12,0.52,0.26
Int_ai,-0.45,0.38,NaN,0.24,0.12,1.00,0.02,0.58
Plausible_hum,-0.29,0.30,NaN,0.06,0.52,0.02,1.00,0.28
Plaus_ai,-0.51,0.32,NaN,0.01,0.26,0.58,0.28,1.00


In [12]:
# df_joint = pd.merge(df_human[['feature_name', 'Interesting?',  'Plausible/Makes Sense?',"COMMENTS"]],
#          df_ai,left_on="feature_name",right_on="Feature",how="inner",suffixes= ('_hum', '_ai'),).drop(columns=["Feature"])

In [13]:
df_joint.select_dtypes(["bool","number"]).corr().round(3)

,novel,plausible,utility,Match Score,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai
novel,1.000,-0.448,NaN,-0.103,-0.046,-0.447,-0.293,-0.511
plausible,-0.448,1.000,NaN,-0.105,0.090,0.381,0.305,0.318
utility,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Score,-0.103,-0.105,NaN,1.000,0.154,0.243,0.062,0.010
Interesting_hum,-0.046,0.090,NaN,0.154,1.000,0.118,0.525,0.255
Int_ai,-0.447,0.381,NaN,0.243,0.118,1.000,0.015,0.582
Plausible_hum,-0.293,0.305,NaN,0.062,0.525,0.015,1.000,0.284
Plaus_ai,-0.511,0.318,NaN,0.010,0.255,0.582,0.284,1.000


In [14]:
df_joint.dtypes

feature             object
target              object
novel                int64
plausible            int64
utility              int64
novel_cot           object
plausible_cot       object
utility_cot         object
Match Score        float64
feature_name        object
Interesting_hum      int64
Int_ai               int64
Plausible_hum        int64
Plaus_ai             int64
COMMENTS            object
dtype: object

In [15]:
df_joint.select_dtypes(["bool","number"]).mean().round(2)

novel               0.83
plausible           0.30
utility             0.00
Match Score        95.85
Interesting_hum     0.57
Int_ai              0.15
Plausible_hum       0.55
Plaus_ai            0.34
dtype: float64

In [16]:
## not sure what happens to missing/-1 cases? Ignore for now

In [17]:
# df_joint_match = df_joint.loc[(df_joint['Interesting?_ai']==df_joint['Interesting?_hum'])\
# | (df_joint['Plausible/Makes Sense?_ai']==df_joint['Plausible/Makes Sense?_hum'])]
# # df_joint.loc[df_joint['Interesting?_ai']>0].select_dtypes(["bool","number"]).corr().round(3)
# print(df_joint_match.shape)
# df_joint_match.select_dtypes(["bool","number"]).corr().round(3)

In [18]:
# df_joint.loc[(df_joint[ 'Interesting_hum']) | (df_joint['Plausible_hum'])]
df_joint.loc[(df_joint[ 'Interesting_hum']>0)].select_dtypes("number").corr().round(2).dropna(how="all").dropna(axis=1,how="all")

,novel,plausible,Match Score,Int_ai,Plausible_hum,Plaus_ai
novel,1.00,-0.27,-0.20,-0.51,-0.25,-0.53
plausible,-0.27,1.00,-0.14,0.47,0.38,0.32
Match Score,-0.20,-0.14,1.00,0.20,-0.16,-0.09
Int_ai,-0.51,0.47,0.20,1.00,0.03,0.53
Plausible_hum,-0.25,0.38,-0.16,0.03,1.00,0.12
Plaus_ai,-0.53,0.32,-0.09,0.53,0.12,1.00


In [19]:
df_joint.loc[(df_joint[ 'Interesting_hum']>0)].select_dtypes("number").agg(["mean","sum"]).round(2)

,novel,plausible,utility,Match Score,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai
mean,0.81,0.33,0.0,96.67,1.0,0.19,0.78,0.44
sum,22.00,9.00,0.0,2610.00,27.0,5.00,21.00,12.00


In [20]:
# df_joint.to_csv("merged_chol_report_mini.csv",index=False)

In [21]:
df_joint.columns

Index(['feature', 'target', 'novel', 'plausible', 'utility', 'novel_cot',
       'plausible_cot', 'utility_cot', 'Match Score', 'feature_name',
       'Interesting_hum', 'Int_ai', 'Plausible_hum', 'Plaus_ai', 'COMMENTS'],
      dtype='object')

In [25]:
df_joint.loc[(df_joint[ 'Interesting_hum']<1)].dropna(subset="COMMENTS")#.tail(10)

,feature,target,novel,plausible,utility,novel_cot,plausible_cot,utility_cot,Match Score,feature_name,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai,COMMENTS
2,(CED) coeliac disease genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between coeliac disease and ga...,Coeliac disease is an autoimmune condition tha...,To assess the utility of the feature '(CED) co...,94.0,Genetic risk for coeliac disease (CED),0,0,0,0,High P value
5,PSO psoriasis genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,1,0,The association between psoriasis (PSO) and ga...,To determine if there is a plausible explanati...,To assess the utility of the feature 'PSO psor...,93.0,Genetic risk for psoriasis (PSO),0,0,0,0,High P value + low correlation
13,SHBG,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The feature 'SHBG' (Sex Hormone Binding Globul...,Sex Hormone Binding Globulin (SHBG) is primari...,To assess the utility of the feature 'SHBG' fo...,100.0,SHBG,0,0,1,1,M<>F
16,EOC epithelial ovarian cancer genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The relationship between epithelial ovarian ca...,To determine if there is a plausible explanati...,To assess the utility of the feature 'EOC epit...,95.0,Genetic risk for epithelial ovarian cancer (EOC),0,0,0,0,Mid P value
17,Standing height,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The relationship between standing height and g...,The question asks whether there is a plausible...,To assess the utility of the feature 'Standing...,100.0,Standing height,0,0,1,0,M<>F
19,POAG primary open angle glaucoma genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The association between primary open-angle gla...,To determine if there is a plausible explanati...,To assess the utility of the feature 'POAG pri...,96.0,Genetic risk for primary open angle glaucoma (...,0,0,0,0,High P value + low correlation
25,Other diseases of respiratory system NEC,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The relationship between respiratory diseases ...,Cholelithiasis (gallstones) and related gallbl...,To assess the utility of the feature 'Other di...,100.0,"Other diseases of respiratory system, NEC",0,0,0,0,High P value + low correlation
28,(AF) atrial fibrillation genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The relationship between atrial fibrillation (...,Atrial fibrillation (AF) is primarily associat...,To assess the utility of the feature '(AF) atr...,95.0,Genetic risk for atrial fibrillation (AF),0,0,0,0,Mid P value
32,Neuroticism score,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The association between neuroticism and gallbl...,The relationship between neuroticism and gallb...,To assess the utility of the 'Neuroticism scor...,100.0,Neuroticism score,0,0,0,0,"The p-value is very high, so I'm nor sure if i..."
34,(PC) prostate cancer genetic risk,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,1,0,0,The relationship between prostate cancer genet...,Prostate cancer and gallbladder diseases such ...,To assess the utility of the feature '(PC) pro...,94.0,Genetic risk for prostate cancer (PC),0,0,0,0,Mid P value + low correlation


In [28]:
df_joint.loc[(df_joint[ 'novel']==0) & (df_joint[ 'Interesting_hum']>0)]

,feature,target,novel,plausible,utility,novel_cot,plausible_cot,utility_cot,Match Score,feature_name,Interesting_hum,Int_ai,Plausible_hum,Plaus_ai,COMMENTS
46,Alcohol drinker status Current,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,0,1,0,The relationship between alcohol consumption a...,The relationship between alcohol consumption a...,To assess the utility of the feature 'Alcohol ...,100.0,Alcohol drinker status Current,1,1,1,1,would expect a a higher correlation
48,"Long-standing illness, disability or infirmity",(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,0,1,0,"The association between long-standing illness,...",Cholelithiasis (gallstones) and related gallbl...,To assess the utility of the feature 'Long-sta...,96.0,"Long-standing illness, disability or infirmity...",1,1,1,1,CVD risk factors
51,Alcohol intake frequency Daily or almost daily,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,0,1,0,The relationship between alcohol intake freque...,The relationship between alcohol intake freque...,To assess the utility of the feature 'Alcohol ...,100.0,Alcohol intake frequency. Daily or almost daily,1,1,1,1,inline with https://pubmed.ncbi.nlm.nih.gov/2...
52,Overall health rating Excellent,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,0,0,0,The feature 'Overall health rating Excellent' ...,Cholelithiasis (gallstones) and related gallbl...,To assess the utility of the feature 'Overall ...,100.0,Overall health rating Excellent,1,0,1,1,healthy life style
53,Testosterone,(Cholelithiasis) OR (Gallstone) OR (Gallbladde...,0,0,0,The relationship between testosterone levels a...,Cholelithiasis (gallstone disease) is more pre...,To assess the utility of the feature 'Testoste...,100.0,Testosterone,1,0,1,1,Women have more
